In [ ]:
!pip install ssspy

In [ ]:
import numpy as np
import scipy.signal as ss
import matplotlib.pyplot as plt
import IPython.display as ipd

In [ ]:
from ssspy.utils.dataset import download_sample_speech_data

In [ ]:
n_sources = 2
max_duration = 10
sisec2010_tag = "dev1_female3"
n_fft, hop_length = 4096, 2048

In [ ]:
waveform_src_img, sample_rate = download_sample_speech_data(
    n_sources=n_sources,
    sisec2010_tag=sisec2010_tag,
    max_duration=max_duration,
    conv=True,
)  # (n_channels, n_sources, n_samples)
waveform_mix = np.sum(waveform_src_img, axis=1)  # (n_channels, n_samples)

In [ ]:
for idx, waveform in enumerate(waveform_mix):
    print("Mixture: {}".format(idx + 1))
    display(ipd.Audio(waveform, rate=sample_rate))
    print()

In [ ]:
import functools

In [ ]:
from ssspy.transform import whiten
from ssspy.algorithm import projection_back
from ssspy.bss.pdsbss import PDSBSS

In [ ]:
def l21_fn(y: np.ndarray) -> np.ndarray:
    """Compute sum of mixed L21 norm.

    Args:
        y (np.ndarray):
            Input vector with shape of (n_sources, n_bins, n_frames).

    Returns:
        Sum of mixed L21 norm.
    """
    G = np.linalg.norm(y, axis=1)
    loss = np.sum(G, axis=(0, 1))

    return loss

def lamb_l1_fn(y: np.ndarray, lamb: float = 1) -> np.ndarray:
    """Compute sum of L1 norm.

    Args:
        y (np.ndarray):
            Input vector with shape of (n_sources, n_bins, n_frames).

    Returns:
        Sum of L1 norm.
    """
    G = np.abs(y)
    loss = lamb * np.sum(G, axis=(0, 1, 2))

    return loss

def prox_l21(y: np.ndarray, step_size: float = 1) -> np.ndarray:
    """Apply proximal operator of mixed L21 norm.

    Args:
        y (np.ndarray):
            Input vector with shape of (n_sources, n_bins, n_frames).
        step_size (float):
            Step size parameter.

    Returns:
        Output value computed by proximal operator of mixed L21 norm.
        The shape of (n_sources, n_bins, n_frames).
    """
    norm = np.linalg.norm(y, axis=1, keepdims=True)

    return np.maximum(1 - step_size / norm, 0) * y

def prox_lamb_l1(y: np.ndarray, step_size: float=1, lamb: float = 1) -> np.ndarray:
    """Apply proximal operator of L1 norm.

    Args:
        y (np.ndarray):
            Input vector with shape of (n_sources, n_bins, n_frames).
        step_size (float):
            Step size parameter.

    Returns:
        Output value computed by proximal operator of L1 norm.
        The shape of (n_sources, n_bins, n_frames).
    """
    norm = np.abs(y)

    return np.maximum(1 - (step_size * lamb) / norm, 0) * y

In [ ]:
# SparseIVA without masking
penalty_fn = [
    l21_fn,
    functools.partial(lamb_l1_fn, lamb=2e-3),
]
prox_penalty = [
    prox_l21,
    functools.partial(prox_lamb_l1, lamb=2e-3),
]

In [ ]:
pds_bss = PDSBSS(
    mu1=1,
    mu2=1,
    alpha=1.75,
    penalty_fn=penalty_fn,
    prox_penalty=prox_penalty,
    scale_restoration=False
)
print(pds_bss)

In [ ]:
_, _, spectrogram_mix = ss.stft(waveform_mix, window="hann", nperseg=n_fft, noverlap=n_fft-hop_length)

In [ ]:
spectrogram_mix_whitened = whiten(spectrogram_mix)
spectrogram_mix_normalized = pds_bss.normalize_by_spectral_norm(spectrogram_mix_whitened)
spectrogram_est = pds_bss(spectrogram_mix_normalized, n_iter=1000)
spectrogram_est = projection_back(spectrogram_est, reference=spectrogram_mix)

In [ ]:
_, waveform_est = ss.istft(spectrogram_est, window="hann", nperseg=n_fft, noverlap=n_fft-hop_length)

In [ ]:
for idx, waveform in enumerate(waveform_est):
    print("Estimated source: {}".format(idx + 1))
    display(ipd.Audio(waveform, rate=sample_rate))
    print()

In [ ]:
plt.figure()
plt.plot(pds_bss.loss[1:])
plt.show()
plt.close()